In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from os import listdir
import glob
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from __future__ import print_function
import os
from scipy import misc
import glob
import matplotlib.pyplot as plt   
import sys 
from scipy.ndimage import rotate
#from scipy.misc import imread, imshow
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import sys
import shutil
import random
import numpy as np
from keras.models import load_model
import pandas as pd
import cv2
from tensorflow.contrib.keras.api.keras.layers import Dropout
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import Conv2D
from tensorflow.contrib.keras.api.keras.layers import MaxPooling2D
from tensorflow.contrib.keras.api.keras.layers import Flatten
from tensorflow.contrib.keras.api.keras.layers import Dense
from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
import os

In [0]:
folder_path = '/content/gdrive/My Drive/AI Exam/Image Classification/Data_Cars and Trucks'

In [0]:
trucks_cars=os.listdir(folder_path)

In [0]:
trucks_cars

['truck', 'car']

In [0]:
truck = os.listdir(folder_path+'/'+trucks_cars[1])

In [0]:
cars = os.listdir(folder_path+'/'+trucks_cars[0])

In [0]:
len(trucks),len(cars)

(392, 402)

In [0]:
trucks_train = trucks[:316]
cars_train = cars[:316]
trucks_test = trucks[316:]
cars_test = cars[316:]

In [0]:
os.mkdir(folder_path+'/train')
os.mkdir(folder_path+'/test')

In [0]:
os.mkdir(folder_path+'/train'+'/cars')
os.mkdir(folder_path+'/train'+'/trucks')
os.mkdir(folder_path+'/test'+'/cars')
os.mkdir(folder_path+'/test'+'/trucks')

In [0]:
for i in range(0,len(trucks_train)):
    image = cv2.imread(folder_path+"/"+trucks_cars[1]+'/'+trucks_train[i]) 
    cv2.imwrite(os.path.join(folder_path+'/train'+'/trucks'+'/'+trucks_train[i]),image)

In [0]:
for j in range(0,len(cars_train)):
    image = cv2.imread(folder_path+"/"+trucks_cars[0]+'/'+cars_train[j])
    cv2.imwrite(os.path.join(folder_path+'/train'+'/cars'+'/'+cars_train[j]),image)

In [0]:
for i in range(0,len(trucks_test)):
    image = cv2.imread(folder_path+"/"+trucks_cars[1]+'/'+trucks_test[i]) 
    cv2.imwrite(os.path.join(folder_path+'/test'+'/trucks'+'/'+trucks_test[i]),image)
    image1 = cv2.imread(folder_path+"/"+trucks_cars[0]+'/'+cars_test[i]) 
    cv2.imwrite(os.path.join(folder_path+'/test'+'/cars'+'/'+cars_test[i]),image1)

In [0]:
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''
 
    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n"\
            .format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1
 
    def on_train_begin(self, logs={}):
        self.losses += 'Training begins...\n'

In [0]:
training_set_path = os.path.join('/content/gdrive/My Drive/AI Exam/Image Classification/Data_Cars and Trucks/train')
test_set_path = os.path.join('/content/gdrive/My Drive/AI Exam/Image Classification/Data_Cars and Trucks/test')

In [0]:
# Initialising the CNN
classifier = Sequential()
 
# Step 1 - Convolution
input_size = (192, 192)
classifier.add(Conv2D(32, (3, 3), input_shape=(*input_size, 3), activation='relu'))
 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))  # 2x2 is optimal
 
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
 
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
 
# Step 4 - Full connection
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=2, activation='softmax'))
 
# Compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# Part 2 - Fitting the CNN to the images
batch_size = 32
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
 
test_datagen = ImageDataGenerator(rescale=1. / 255)
 
training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
 
test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            class_mode='categorical')
 

Found 632 images belonging to 2 classes.
Found 152 images belonging to 2 classes.


In [0]:
# Create a loss history
history = LossHistory()
 
classifier.fit_generator(training_set,
                         steps_per_epoch=632,
                         epochs=50,
                         validation_data=test_set,
                         validation_steps=152,
                         workers=12,
                         callbacks=[history])
 

Epoch 1/50
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
632/632 [==============================] - 193s 305ms/step - loss: 0.6351 - acc: 0.6150 - val_loss: 0.8163 - val_acc: 0.6181
Epoch 2/50
632/632 [==============================] - 190s 301ms/step - loss: 0.3562 - acc: 0.8335 - val_loss: 1.4882 - val_acc: 0.6510
Epoch 3/50
632/632 [==============================] - 187s 295ms/step - loss: 0.1333 - acc: 0.9501 - val_loss: 1.9456 - val_acc: 0.6726
Epoch 4/50
632/632 [==============================] - 189s 300ms/step - loss: 0.0724 - acc: 0.9756 - val_loss: 2.5835 - val_acc: 0.6581
Epoch 5/50
632/632 [==============================] - 188s 298ms/step - loss: 0.0425 - acc: 0.9856 - val_loss: 2.9931 - val_acc: 0.6313
Epoch 6/50
632/632 [==============================] - 187s 297ms/step - loss: 0.0339 - acc: 0.9894 - val_loss: 2.7097 - val_acc: 0.6254
Epoch 7/50
632/632 [==============================] - 189s 299ms/step - loss: 0.0360 - acc